# Semana 8 — Aprendizado Não Supervisionado

Este notebook explora clustering (K-means e DBSCAN), redução de dimensionalidade com PCA e um autoencoder simples para reconstrução de dados.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons, load_digits
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

plt.style.use('seaborn-v0_8')
np.random.seed(42)


## Dados 2D para clustering

Usamos `make_moons` para gerar dois grupos não-lineares com ruído.

In [ ]:
X, _ = make_moons(n_samples=500, noise=0.08, random_state=42)

fig, ax = plt.subplots(figsize=(5, 4))
ax.scatter(X[:, 0], X[:, 1], s=20, alpha=0.6)
ax.set_title('Dados 2D (make_moons)')
plt.show()


## K-means vs. DBSCAN

Comparamos K-means (assume clusters esféricos) com DBSCAN (baseado em densidade).

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=42)
labels_km = kmeans.fit_predict(X)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# DBSCAN costuma precisar de dados escalados
labels_db = DBSCAN(eps=0.35, min_samples=8).fit_predict(X_scaled)

fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharey=True)
axes[0].scatter(X[:, 0], X[:, 1], c=labels_km, cmap='viridis', s=20)
axes[0].set_title('K-means (k=2)')

axes[1].scatter(X[:, 0], X[:, 1], c=labels_db, cmap='viridis', s=20)
axes[1].set_title('DBSCAN')
plt.show()

# Métricas de silhueta (quando há mais de um cluster)
if len(set(labels_km)) > 1:
    print('Silhueta K-means:', silhouette_score(X, labels_km))
if len(set(labels_db)) > 1 and -1 not in set(labels_db):
    print('Silhueta DBSCAN:', silhouette_score(X, labels_db))


## PCA em dados de alta dimensionalidade (Digits)

Aplicamos PCA para reduzir 64 dimensões para 2 e visualizar a separação dos dígitos.

In [ ]:
digits = load_digits()
X_digits = digits.data

pca = PCA(n_components=2, random_state=42)
X_2d = pca.fit_transform(X_digits)

fig, ax = plt.subplots(figsize=(6, 5))
scatter = ax.scatter(X_2d[:, 0], X_2d[:, 1], c=digits.target, cmap='tab10', s=15)
ax.set_title('PCA (2D) nos dígitos')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
legend = ax.legend(*scatter.legend_elements(num=10), title='Dígitos', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()


## K-means com e sem PCA (exercício guiado)

Aplicamos K-means diretamente nos dados e depois em uma versão reduzida por PCA (30 componentes) para comparar silhueta e tempo de treinamento.

In [ ]:
import time

kmeans_raw = KMeans(n_clusters=10, random_state=42)
start = time.time()
labels_raw = kmeans_raw.fit_predict(X_digits)
raw_time = time.time() - start

pca_30 = PCA(n_components=30, random_state=42)
X_pca30 = pca_30.fit_transform(X_digits)

kmeans_pca = KMeans(n_clusters=10, random_state=42)
start = time.time()
labels_pca = kmeans_pca.fit_predict(X_pca30)
pca_time = time.time() - start

sil_raw = silhouette_score(X_digits, labels_raw)
sil_pca = silhouette_score(X_pca30, labels_pca)

pd.DataFrame([
    {'metodo': 'K-means raw', 'silhueta': sil_raw, 'tempo_s': raw_time},
    {'metodo': 'K-means + PCA(30)', 'silhueta': sil_pca, 'tempo_s': pca_time}
])


## Autoencoder simples (Keras)

Implementamos um autoencoder denso para reconstruir os dígitos. O objetivo é reduzir dimensionalidade aprendendo uma representação comprimida.

> **Observação:** Execute esta célula apenas se `tensorflow/keras` estiver disponível no ambiente.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Normalização dos dígitos
X_auto = X_digits / 16.0

input_dim = X_auto.shape[1]
encoding_dim = 16

input_layer = keras.Input(shape=(input_dim,))
encoded = layers.Dense(32, activation='relu')(input_layer)
encoded = layers.Dense(encoding_dim, activation='relu')(encoded)
decoded = layers.Dense(32, activation='relu')(encoded)
decoded = layers.Dense(input_dim, activation='sigmoid')(decoded)

autoencoder = keras.Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

history = autoencoder.fit(
    X_auto, X_auto,
    epochs=30,
    batch_size=64,
    validation_split=0.2,
    verbose=0
)

recon = autoencoder.predict(X_auto[:10])
recon_error = np.mean((X_auto[:10] - recon) ** 2)
print(f"Erro médio de reconstrução (10 amostras): {recon_error:.4f}")


## Resumo

- K-means funciona bem em clusters convexos, mas falha em formas não lineares; DBSCAN captura densidade e detecta ruído.
- PCA ajuda a reduzir custo computacional e pode melhorar separação em clusters quando remove ruído.
- Autoencoders fornecem uma representação compacta para reconstrução de dados, útil para redução de dimensionalidade não linear.